In [3]:
import pandas as pd
import numpy as np
from pandas import Series

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [4]:
train_df = pd.read_csv('/kaggle/input/layer12/train.csv')
valid_df = pd.read_csv('/kaggle/input/layer12/valid.csv')
test_df = pd.read_csv('/kaggle/input/layer12/test.csv')

In [5]:
train_df.shape

(28520, 772)

In [6]:
train_df.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_763,feature_764,feature_765,feature_766,feature_767,feature_768,label_1,label_2,label_3,label_4
0,0.031138,0.079892,0.157382,-0.014636,-0.051778,-0.021332,-0.073593,-0.005386,-0.212557,0.099683,...,-0.085248,-0.096007,-0.000766,0.021399,-0.041432,0.094806,45,NaN,1,6
1,0.113040,0.175731,0.217741,-0.196254,-0.010129,-0.030586,0.067114,-0.072412,-0.239192,0.104741,...,-0.090283,-0.053885,-0.010967,0.062209,-0.122958,0.192949,45,NaN,1,6
2,0.048570,0.091281,0.160776,-0.150937,0.020115,0.044117,-0.050092,-0.045661,-0.155332,0.117206,...,-0.021524,-0.008411,-0.006248,0.031468,-0.056915,0.154731,45,NaN,1,6
3,0.039212,0.118388,0.173831,-0.096659,-0.008702,0.061298,0.008974,-0.003277,-0.065046,0.095480,...,-0.071936,-0.023120,-0.007812,0.057600,-0.121892,0.072796,45,NaN,1,6
4,0.056019,0.170639,0.157917,-0.228605,-0.065965,-0.088732,-0.082243,-0.080568,-0.341500,0.142430,...,-0.155621,-0.079447,0.015316,0.127726,-0.151966,0.169634,45,NaN,1,6


In [7]:
missing_columns = train_df.columns[train_df.isnull().any()]
missing_counts = train_df[missing_columns].isnull().sum()

print('Missing Columns and Counts')
for column in missing_columns:
    print( str(column) +' : '+ str(missing_counts[column]))

Missing Columns and Counts
label_2 : 480


In [8]:
train_data = train_df.copy()
valid_data = valid_df.copy()
test_data = test_df.copy()

In [9]:
train_df.describe()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_763,feature_764,feature_765,feature_766,feature_767,feature_768,label_1,label_2,label_3,label_4
count,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,...,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28040.000000,28520.000000,28520.000000
mean,0.042487,0.068749,0.145547,-0.070646,-0.013539,0.003395,-0.041282,-0.028283,-0.106602,0.053686,...,-0.022102,-0.044743,-0.004380,0.049072,-0.028722,0.075717,30.498843,27.975107,0.799299,5.997125
std,0.048918,0.046354,0.065332,0.046671,0.027635,0.031248,0.026479,0.029632,0.070775,0.030945,...,0.053250,0.031361,0.025829,0.050536,0.032622,0.044879,17.328389,5.735913,0.400532,2.375567
min,-0.079594,-0.062608,-0.048545,-0.307243,-0.178347,-0.194771,-0.197551,-0.304828,-0.421257,-0.049723,...,-0.253255,-0.264549,-0.137827,-0.117697,-0.302399,-0.090777,1.000000,22.000000,0.000000,0.000000
25%,0.009979,0.037225,0.100677,-0.088834,-0.027810,-0.010617,-0.056682,-0.044344,-0.135110,0.033236,...,-0.042332,-0.056918,-0.018848,0.012599,-0.045226,0.045309,15.000000,25.000000,1.000000,6.000000
50%,0.024445,0.056119,0.123554,-0.057386,-0.011423,0.006173,-0.041501,-0.025805,-0.080715,0.045567,...,-0.007960,-0.037407,-0.004701,0.033121,-0.022919,0.064875,30.000000,27.000000,1.000000,6.000000
75%,0.058410,0.086358,0.173234,-0.039661,0.001544,0.021250,-0.026559,-0.009324,-0.057873,0.065670,...,0.012116,-0.024179,0.010218,0.072599,-0.006335,0.097642,46.000000,30.000000,1.000000,6.000000
max,0.274146,0.332288,0.454182,0.059362,0.196950,0.213127,0.124194,0.105714,0.192121,0.252320,...,0.209455,0.054555,0.215375,0.376414,0.125857,0.416291,60.000000,61.000000,1.000000,13.000000


In [10]:
from sklearn.preprocessing import RobustScaler # eliminate outliers

x_train = {}
x_valid = {}
x_test = {}

y_train = {}
y_valid = {}
y_test = {}

#create dictionaries for each label
for target_label in ['label_1','label_2','label_3','label_4']:

  if target_label == "label_2":
    train = train_df[train_df['label_2'].notna()]
    valid = valid_df[valid_df['label_2'].notna()]
  else:
    train = train_df
    valid = valid_df

  test = test_df

  scaler = RobustScaler()

  x_train[target_label] = pd.DataFrame(scaler.fit_transform(train.drop(['label_1','label_2','label_3','label_4'], axis=1)), columns=[f'feature_{i}' for i in range(1,769)])
  y_train[target_label] = train[target_label]

  x_valid[target_label] = pd.DataFrame(scaler.transform(valid.drop(['label_1','label_2','label_3','label_4'], axis=1)), columns=[f'feature_{i}' for i in range(1,769)])
  y_valid  [target_label] = valid[target_label]

  x_test[target_label] = pd.DataFrame(scaler.transform(test.drop(["ID"],axis=1)), columns=[f'feature_{i}' for i in range(1,769)])

In [11]:
x_train_df = x_train['label_1'].copy()
y_train_df = y_train['label_1'].copy()

x_valid_df = x_valid['label_1'].copy()
y_valid_df = y_valid['label_1'].copy()

x_test_df = x_test['label_1'].copy()

# Cross Validation

In [12]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, KFold

# Perform cross-validation
scores = cross_val_score(SVC(), x_train_df, y_train_df, cv=5, scoring='accuracy')

mean_accuracy = scores.mean()
std_accuracy = scores.std()
# Print the cross-validation scores
print('Support Vector Machines')
print('\n')
print("Cross-validation scores:", scores)
print(f"Mean Accuracy: {mean_accuracy:.2f}")
print(f"Standard Deviation: {std_accuracy:.2f}")

Support Vector Machines


Cross-validation scores: [0.89323282 0.89814165 0.89270687 0.88709677 0.89130435]
Mean Accuracy: 0.89
Standard Deviation: 0.00


# Feature Engineering

In [13]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.975, svd_solver='full')
pca.fit(x_train_df)
x_train_df_pca = pd.DataFrame(pca.transform(x_train_df))
x_valid_df_pca = pd.DataFrame(pca.transform(x_valid_df))
x_test_df_pca = pd.DataFrame(pca.transform(x_test_df))
print('Shape after PCA: ',x_train_df_pca.shape)

Shape after PCA:  (28520, 245)


In [14]:
from sklearn import metrics

# SVM

In [15]:
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

classifier = svm.SVC(kernel='linear', C=1)

classifier.fit(x_train_df_pca, y_train_df)

y_valid_pred = classifier.predict(x_valid_df_pca)

print("acc_score: ",metrics.accuracy_score(y_valid_df, y_valid_pred))

acc_score:  0.08533333333333333


In [16]:
# from sklearn.svm import SVC
# from sklearn.model_selection import RandomizedSearchCV
# from scipy.stats import uniform, randint
# import numpy as np

# param_dist = {
#     'C': [100,10,1,0,0.1,0.01],
#     'kernel': ['rbf','linear','poly','sigmoid'],
#     'gamma': ['scale','auto'],
#     'degree': [1,2,3,4],
#     'class_weight' : ['none','balanced']
# }

# svm = SVC()

# random_search = RandomizedSearchCV(
#     svm, param_distributions=param_dist, n_iter=10, cv=5, n_jobs=-1, random_state=42, scoring='accuracy'
# )

# random_search.fit(x_train_df_pca, y_train_df)

# best_params = random_search.best_params_
# best_model = random_search.best_estimator_

# print("best parameters:", best_params)

Label 1 
best parameters: {'kernel': 'rbf', 'gamma': 'scale', 'degree': 4, 'class_weight': 'balanced', 'C': 100}


In [17]:
from sklearn import svm

classifier = svm.SVC(kernel='rbf', C=100, gamma='scale', degree=4, class_weight='balanced')

classifier.fit(x_train_df_pca, y_train_df)

y_valid_pred = classifier.predict(x_valid_df_pca)

print("acc_score: ",metrics.accuracy_score(y_valid_df, y_valid_pred))

y_test_predict_after_pca = classifier.predict(x_test_df_pca)



acc_score:  0.018666666666666668


# RandomForest

In [18]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=100, random_state=42)

classifier.fit(x_train_df, y_train_df)

y_valid_pred = classifier.predict(x_valid_df)

print("accuracy_score: ",metrics.accuracy_score(y_valid_df, y_valid_pred))

y_test_pred = classifier.predict(x_test_df)

accuracy_score:  0.052


# CSV Creation

In [19]:
output_df=pd.DataFrame(columns=["ID","label_1","label_2","label_3","label_4"])

In [20]:
IDs = list(i for i in range(1, len(test_df)+1))
output_df["ID"] = IDs

In [21]:
output_df["label_1"] = y_test_predict_after_pca

In [22]:
output_df

,ID,label_1,label_2,label_3,label_4
0,1,26,NaN,NaN,NaN
1,2,18,NaN,NaN,NaN
2,3,16,NaN,NaN,NaN
3,4,7,NaN,NaN,NaN
4,5,58,NaN,NaN,NaN
...,...,...,...,...,...
739,740,35,NaN,NaN,NaN
740,741,35,NaN,NaN,NaN
741,742,54,NaN,NaN,NaN
742,743,38,NaN,NaN,NaN


In [23]:
output_df.to_csv('/kaggle/working/output12_1.csv',index=False)